In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (initialize, load_generation_data, generate_randomized_generation,
generate_rt_scenarios,load_market_prices)

In [42]:
generation_data, I, T = load_generation_data(date_filter="2022-07-18")
P_DA, P_PN = load_market_prices("data/price/20220718da.csv")
R, P_RT, K, K0, M = initialize(I, T, generation_data)

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv


TypeError: initialize() missing 1 required positional argument: 'ny_rt'

In [27]:
# def load_generation_data(include_files=None, date_filter=None):
#     data_dir = "/Users/jangseohyun/Documents/workspace/symply/DER/data/generation"
#     all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

#     if include_files is not None:
#         for file in include_files:
#             if file not in all_files:
#                 raise ValueError(f"파일을 찾을 수 없습니다: {file}")
#         files_to_load = [f for f in all_files if f in include_files]
#     else:
#         files_to_load = all_files

#     I = len(files_to_load)  
#     T = 24 
#     generation_data = np.zeros((I, T)) 

#     loaded_files = []
    
#     for idx, file in enumerate(files_to_load):
#         file_path = os.path.join(data_dir, file)
#         df = pd.read_csv(file_path)
#         df.columns = df.columns.str.strip()

#         date_col = "Date"
#         hour_col = "Hour (Eastern Time, Daylight-Adjusted)"
#         gen_col = "Electricity Generated"

#         if any(col not in df.columns for col in [date_col, hour_col, gen_col]):
#             print(f"{file}: 필요한 컬럼 없음. 스킵됨.")
#             continue

#         if date_filter:
#             df = df[df[date_col] == date_filter]
#             if df.empty:
#                 print(f"{file}: {date_filter} 데이터 없음. 스킵됨.")
#                 continue 

#         df = df[df[hour_col].astype(str).str.match(r'^\d+$')]
#         df["Time"] = df[hour_col].astype(int)
#         df = df[df["Time"].between(0, 23)]

#         for t in range(T):
#             if t in df["Time"].values:
#                 generation_data[idx, t] = df[df["Time"] == t][gen_col].values[0]

#         loaded_files.append(file)

#     print(f"✅ 총 {I}개 파일을 불러왔습니다: {', '.join(loaded_files)}")

#     return generation_data, I, T

# def generate_randomized_generation(I, T, S, data, randomness_level):
#     np.random.seed(1)

#     noise_ranges = {
#         "low": (0.8, 1.2),
#         "medium": (0.5, 1.5),
#         "high": (0.2, 1.8),
#     }

#     if randomness_level not in noise_ranges:
#         raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")

#     low, high = noise_ranges[randomness_level]
#     noise_factors = np.random.uniform(low, high, size=(I, T, S))

#     generation_r = np.expand_dims(data, axis=-1) * noise_factors
    
#     print(f"📊 데이터 Shape: I={I}, T={T}, S={S}")
#     return generation_r


# def generate_rt_scenarios(rt_da, S, randomness_level):

#     rt_da["Time Stamp"] = pd.to_datetime(rt_da["Time Stamp"])
#     nyc_rt = rt_da[rt_da["Name"] == "N.Y.C."].copy() 

#     # Extract the start of the day and filter only the first 24 hours
#     start_of_day = nyc_rt["Time Stamp"].min().floor("D")
#     end_of_day = start_of_day + pd.Timedelta(hours=23)
#     nyc_rt = nyc_rt[(nyc_rt["Time Stamp"] >= start_of_day) & (nyc_rt["Time Stamp"] <= end_of_day)]

#     nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")
#     hourly_avg = nyc_rt.groupby("Hour")["LBMP ($/MWHr)"].mean().reset_index()
#     price_hourly = hourly_avg["LBMP ($/MWHr)"].to_numpy()
#     T = len(price_hourly)

#     np.random.seed(11)
#     noise_ranges = {
#         "low": (0.95, 1.05),
#         "medium": (0.85, 1.15),
#         "high": (0.7, 1.3),
#     }

#     if randomness_level not in noise_ranges:
#         raise ValueError("Invalid randomness level. Choose from 'low', 'medium', 'high'.")

#     low, high = noise_ranges[randomness_level]
#     noise_factors = np.random.uniform(low, high, size=(T, S))

#     P_RT = np.expand_dims(price_hourly, axis=-1) * noise_factors

#     return P_RT

    
# only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
# ny_da = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718da.csv")
# ny_rt = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718rt.csv")
# ny_da["Time Stamp"] = pd.to_datetime(ny_da["Time Stamp"])
# ny_da["Hour"] = ny_da["Time Stamp"].dt.hour
# nyc_data = ny_da[ny_da["Name"] == "N.Y.C."]
# P_DA = nyc_data["LBMP ($/MWHr)"].to_numpy() * 1.3
# P_PN = P_DA * 1.5

# # plot_generation_data(generation_data, 10)
# # plot_randomized_generation(R, I, T, 7)
# # plot_scenarios_for_generator(R, 1)
# # plot_rt_scenarios(P_RT)


In [28]:
# include_files = ['1201.csv', '137.csv', '281.csv', '397.csv', '401.csv', '430.csv', '514.csv', '524.csv', '775.csv', '89.csv']
include_files = ['1201.csv', '137.csv', '401.csv', '524.csv', '89.csv']
generation_data, I, T = load_generation_data(include_files, "2022-07-18")

S = 10
randomness_level = "high"

R = generate_randomized_generation(I, T, S, generation_data, randomness_level)
P_RT = generate_rt_scenarios(ny_rt, S, randomness_level)
K = np.full(I, 100)
K0 = np.full(I, 10)
M = np.maximum(R, K[:, None, None]).max()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=10


/var/folders/2_/9m9qjphn0xs45xzpvf1hl0j00000gn/T/ipykernel_38273/3674290639.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


In [29]:
only = gp.Model("only")
# only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z")
zc = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="zeta")
delta = only.addVars(I, T, S, vtype=GRB.BINARY, name="delta")

only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i, t in product(range(I), range(T))) \
    + gp.quicksum(1 / S * (P_RT[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i, t, s in product(range(I), range(T), range(S)))
only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    only.addConstr(R[i, t, s] + zd[i, t, s] >= yp[i, t, s] + zc[i, t, s])
    only.addConstr(ym[i, t, s] <= M * delta[i, t, s])
    only.addConstr(zc[i, t, s] <= M * (1 - delta[i, t, s]))
    only.addConstr(zc[i, t, s] <= M * zeta[i, t, s])
    only.addConstr(zd[i, t, s] <= M * (1 - zeta[i, t, s]))
    only.addConstr(z[i, t, s] <= K[i])
    only.addConstr(z[i, t, s] >= 0)
for i, s in product(range(I), range(S)):
    only.addConstr(z[i, 0, s] == K0[i] + zc[i, 0, s] - zd[i, 0, s])
for i, t, s in product(range(I), range(1, T), range(S)):
    only.addConstr(z[i, t, s] == z[i, t - 1, s] + zc[i, t, s] - zd[i, t, s])

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10800 rows, 8520 columns and 26350 nonzeros
Model fingerprint: 0x117b769b
Variable types: 6120 continuous, 2400 integer (2400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 8e+02]
Presolve removed 9480 rows and 7116 columns
Presolve time: 0.07s
Presolved: 1320 rows, 1404 columns, 3880 nonzeros
Variable types: 1064 continuous, 340 integer (340 binary)
Found heuristic solution: objective 1675095.9983

Root relaxation: objective 1.709055e+06, 792 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1709055.42    0    2 16750

In [30]:
da_profit = sum(P_DA[t] * x[i, t].X for i in range(I) for t in range(T))
rt_profit = sum(P_RT[t, s] * yp[i, t, s].X / S for i in range(I) for t in range(T) for s in range(S))
pn_cost   = sum(P_PN[t] * ym[i, t, s].X / S for i in range(I) for t in range(T) for s in range(S))
total_profit = da_profit + rt_profit - pn_cost

print("[DISAGGREGATION MODEL]")
print(f"DA Profit      = {da_profit:.2f}")
print(f"RT Profit      = {rt_profit:.2f}")
print(f"Penalty Cost   = {pn_cost:.2f}")
print(f"Total Profit   = {total_profit:.2f}")

[ONLY MODEL]
DA Profit      = 200157.14
RT Profit      = 1515686.89
Penalty Cost   = 7078.31
Total Profit   = 1708765.71


In [33]:
print("\n[ONLY MODEL] Day-Ahead Commitment (sum over DERs):")
total_only_commit = 0
for t in range(T):
    commit_t = sum(x[i, t].X for i in range(I))
    total_only_commit += commit_t
    print(f"Time {t}: {commit_t:.2f}")
print(f"TOTAL: {total_only_commit:.2f}")


[ONLY MODEL] Day-Ahead Commitment (sum over DERs):
Time 0: 0.00
Time 1: 0.00
Time 2: 0.00
Time 3: 0.00
Time 4: 0.00
Time 5: 0.00
Time 6: 0.00
Time 7: 0.00
Time 8: 0.00
Time 9: 48.21
Time 10: 226.18
Time 11: 0.00
Time 12: 0.00
Time 13: 0.00
Time 14: 383.37
Time 15: 0.00
Time 16: 239.75
Time 17: 0.00
Time 18: 0.00
Time 19: 342.65
Time 20: 109.07
Time 21: 32.50
Time 22: 0.00
Time 23: 0.00
TOTAL: 1381.73
